In [1]:
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient
from typing import Any
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()

tavily_client = TavilyClient()

In [2]:
@tool
def web_search(query: str) -> dict[str, Any]:
    """Search the web for information"""
    return tavily_client.search(query)

In [3]:
SYSTEM_PROMPT = """
You are a personal chef. The user will give you a list of ingredients they have left over in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients they have.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.
"""

In [4]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)
agent = create_agent(
    model=model, tools=[web_search], system_prompt=SYSTEM_PROMPT, checkpointer=InMemorySaver()
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="""I have some leftover chicken and rice. What can I make?
                               Provide me brief instructions for all the recipes.""")]},
    config
)

print(response['messages'][-1].content)

[{'type': 'text', 'text': 'Here are a couple of ideas for your leftover chicken and rice:\n\n**1. Chicken Fried Rice**\n*   Heat a little oil in a pan or wok.\n*   Add your cooked chicken and rice, along with any other vegetables you have (peas, carrots, onions work well).\n*   Stir-fry for a few minutes.\n*   Push the ingredients to one side, add a beaten egg to the other side and scramble it.\n*   Mix everything together and season with soy sauce, a dash of sesame oil, and salt and pepper to taste.\n\n**2. Chicken and Rice Casserole**\n*   Preheat your oven.\n*   In a baking dish, combine your cooked chicken and rice.\n*   You can add a can of cream of mushroom or chicken soup, a splash of milk, and some cooked vegetables (like broccoli or peas).\n*   Stir everything together and top with cheese or breadcrumbs if desired.\n*   Bake until heated through and bubbly, about 20-25 minutes.', 'extras': {'signature': 'CrEBAXLI2nyZ4EkH8WTIeghPGcEMOpAFGlBW8ww/OtV/sCH2A5m4XVYYcyhnQCqvkOuXV/m7m

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?\n                               Provide me brief instructions for all the recipes.', additional_kwargs={}, response_metadata={}, id='d1cea55c-9680-43bb-a3e4-cb1f6deb09f8'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "recipes with leftover chicken and rice"}'}, '__gemini_function_call_thought_signatures__': {'a7fe1b39-491c-4357-be44-bdaec6ca4f09': 'CswBAXLI2nzYnIwV5261ziabDvm+h1X56zUdMtvWYD7dHV81bwxTfARcf0aqmFekCfMv5GSXsCSEazg2R5tfp/NvtybK+JSLdzz+FTDmP6we+0uKGCdl3XnNUysa8in2IUDjcQNyNxkzhV6npoyMrY6JT48X2m/SCng/lTGF814LubNt9nNAdB8iPqRUPRxtWc9SJEBF5Q8OwawsBw87dsYbJ7sQwBOluTRg2fesV/WImYAQN2nm+CLUHqYzMpk8f/Nsa1QoBb8ooI0QTNrl'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bf7f6-3e3c-7362-ad8a-467e54fbe163-0', tool_calls=[{

Trace: https://smith.langchain.com/public/ac827f2f-5e34-45d0-b684-ba74c2374a8b/r